# Data Programming - Numpy

## Similar Movies Exercise  
### Introduction and Intuitions
Suppose that we have a large data set of movie ratings from such as Watcha or Netflix. Is there any way to recommend __'similar'__ movies for customers? For example, let's compare two cases below:

__CASE 1:__ 4 user rated both `Star Wars` and `Lord of Rings` like below:
> USER ID 220 rated both _Star Wars_ and _Lord of Rings_ 5.  
> USER ID 193 rated _Star Wars_ as 4 and _Lord of Rings_ as 5.  
> USER ID 431 rated both _Star Wars_ and _Lord of Rings_ 1.  
> USER ID 940 rated _Star Wars_ as 3 and _Lord of Rings_ as 2.   

We can say that `Star Wars` and `Lord of Rings` are __similar__ movies. Why? Because majority of people rated similar score to both movies. Let's say __similarity of two movies are close to 1__.

__CASE 2:__ 3 user rated both `Home Alone` and `Bohemian Rapsody` like below:
> USER ID 199 rated _Home Alone_ as 4 but _Bohemian Rapsody_ as 1.  
> USER ID 320 rated _Home Alone_ as 2 and _Bohemian Rapsody_ as 5.   
> USER ID 878 rated _Home Alone_ as 5 and _Bohemian Rapsody_ as 1. 

We can say that `Home Alone` and `Bohemian Rapsody` are __not similar__ to each other. Why? Because majority of people rated totally different score to each movies. Let's say __similarity of two movies are close to 0__.

If you watched the `Star Wars` and liked it, it's highly likely to enjoy `Lord of Rings` too. If you watched the `Home Alone` and didn't enjoy it, recommendation algorithm may recommend you `Bohemian Rapsody`, because it's totally different to the former one. If you want to know similar movies of `Star Wars`, algorithm can search for you __movies that have similarity close to 1__. 

There is a movie rating file in __'../data/ml-100k/u.data'__.   
Each line of the file contists of 4 numbers and '\t' charactors between the numbers.  
>`First Number` is __User ID__ who rated the movie.  
>`Second Number` is Movie ID which is rated.  
>`Third Number` is the rating of the movie.  
>`Forth Number` is time stamp which we don't need to care in this exercise.  

Write your code that load the rating data file and make a __integer list__ of (User ID, Movie ID, rating) triples.  Name the list __`ratingList`__.

In [1]:
import pandas as pd

df = pd.read_csv('u.data', sep='\t', names=['User ID', 'Movie ID', 'rating'], usecols=[0, 1, 2])
ratingList = df.values.tolist()

If your code is well designed, run below cell. The output should be [32, 294, 3].

In [2]:
print(ratingList[100])

[32, 294, 3]


By here, 'ratingList' is a 2D list which is size of [Number of Ratings][3].  
Convert this list to Numpy 2D Array and named it as "ratings" that each row constists of (UserID, MovieID, Rating) and check the shape of numpy function __shape__.

In [3]:
import numpy as np

ratings = np.array(ratingList)

By using 'ratings' array, construct __`userRating`__ that userRating[i][j] represents rating of movie ID j by user ID i. If user i didn't rate movie j, then make the rate value as 0.

In [4]:
# Initialize userRating array with zeros
userRating = np.zeros((np.max(ratings, 0)[0] + 1, np.max(ratings, 0)[1] + 1))

for rating in ratings:
    user_id, movie_id, value = rating
    userRating[user_id][movie_id] = value

userRating

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 3., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       ...,
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 5., ..., 0., 0., 0.]])

Find the movies that 'User ID = 6' rated. Print 10 movie IDs in ascending order from the least movie ID.  

The output should be 1, 7, 8, 9, 12, 13, 14, 15, 19, 21

In [5]:
user_6_ratings = userRating[6, :]

# Get the indices of non-zero ratings
rated_movie_indices = np.nonzero(user_6_ratings)[0]

# Sort the movie indices in ascending order
sorted_movie_indices = list(np.sort(rated_movie_indices))

# Print the top 10 movie IDs
print("Top 10 movie IDs rated by User ID 6 in ascending order:")
print(sorted_movie_indices[:10])

Top 10 movie IDs rated by User ID 6 in ascending order:
[1, 7, 8, 9, 12, 13, 14, 15, 19, 21]


Using Numpy, We can calculate the similarity as known as `cosine similarity`.  
For given two vectors of Attributes A and B -in this case, it's movies' ratings vectors.- the similarity is caculated as below fomula.
<img src="./ml-100k/cosineSim.png.">  
  
  
Make a function `computeCosineSimilarity(userRating, movieX, movieY)` which returns pair of `(cosine similarity, number of common pairs)`. Cosine similarity is about movies which IDs are movieX and movieY by their own ratings. (Hint1: You can user 'sqrt' in 'math' library) (Hint2: Make sure you don't devide number by 0)

In [6]:
from math import sqrt

def computeCosineSimilarity(userRating, movieX, movieY):
    
    # Find each movie X and Y's rating list
    ratings_X = userRating[:, movieX]
    ratings_Y = userRating[:, movieY]

    # Find common pairs of non-zero ratings for movies X and Y
    common_pairs = np.logical_and(ratings_X != 0, ratings_Y != 0)

    # Check if there are any common pairs
    if not np.any(common_pairs):
        return (0, 0)

    # For every user, compute ratings mult and square, and then Sum them
    ratings_mult_sum = np.sum(ratings_X[common_pairs] * ratings_Y[common_pairs])
    ratings_X_square_sum = np.sum(ratings_X[common_pairs] ** 2)
    ratings_Y_square_sum = np.sum(ratings_Y[common_pairs] ** 2)

    # Calculate Cosine similarity but be careful not to divide by zero
    denominator = sqrt(ratings_X_square_sum) * sqrt(ratings_Y_square_sum)
    if denominator == 0:
        return (0, 0)

    score = ratings_mult_sum / denominator
    numPairs = np.sum(common_pairs)
    
    return (score, numPairs)

You can check your function by running the code block below.  
This function should return (0.948737, 104).

In [7]:
result = computeCosineSimilarity(userRating, 1,2)
print((round(result[0],6), result[1]))

(0.948737, 104)


__Now__, find top k similar movies with given movie ID.  
Make a function __`similarMovie(userRating, movie, scoreThreshold,coOccurenceThreshold)`__ that returns list of similar movie IDs in descending order of similarities and number of common pairs.   
Sorting code is already in the code block, just implement making a tuple list (similarity) which consists of __`[(movieID, similarity, numPairs)]`__

In [8]:
from operator import itemgetter
def similarMovie(userRating, movie, scoreThreshold = 0.97, coOccurrenceThreshold = 50):
    similarity = []
    
    # For all movie IDs except input 'movie' as ID
    for other_movie in range(1, userRating.shape[1]):
        if other_movie != movie:
            # Calculate the Cosine similarity and number of pairs
            score, numPairs = computeCosineSimilarity(userRating, movie, other_movie)

            # Only if both cosine similarity and pairs are larger than Threshold,
            # Put (movieID, similarity, numPairs) tuple in similarity list
            if score > scoreThreshold and numPairs > coOccurrenceThreshold:
                similarity.append((other_movie, score, numPairs))
    
    # Sort this list by similarity and co-occurency
    rank_similarity = sorted(similarity, key = itemgetter(1, 2), reverse = True)

    return rank_similarity

Run the next block to check your function works appropriate or not.  
It will show list of top similar movies with "Toy Story (1995)".  

In [9]:
nameDict = {}
def loadMovieNames():
    movieNames = {}
    with open("u.item", encoding='ascii', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

nameDict = loadMovieNames()

rank = similarMovie(userRating, 1)
print("Top similar movies with " + nameDict[1] + "are:")
count = 1
for movieID, sim, pairs in rank:
    print("rank " + str(count) + ": " + nameDict[movieID])
    count += 1

Top similar movies with Toy Story (1995)are:
rank 1: Hamlet (1996)
rank 2: Raiders of the Lost Ark (1981)
rank 3: Cinderella (1950)
rank 4: Winnie the Pooh and the Blustery Day (1968)
rank 5: Cool Hand Luke (1967)
rank 6: Great Escape, The (1963)
rank 7: African Queen, The (1951)
rank 8: Apollo 13 (1995)
rank 9: 12 Angry Men (1957)
rank 10: Wrong Trousers, The (1993)
rank 11: Indiana Jones and the Last Crusade (1989)
rank 12: Lion King, The (1994)
rank 13: Glory (1989)
rank 14: Schindler's List (1993)
